In [58]:
import itertools
import sys
import datetime

# Our numerical workhorses
import numpy as np
import pandas as pd
import scipy.integrate

# Import Altair for high level plotting
import altair as alt
import altair_catplot as altcat

# Import Bokeh modules for interactive plotting
import bokeh.io
import bokeh.plotting

# Set up Bokeh for inline viewing
bokeh.io.output_notebook()

# Pevent bulky altair plots
alt.data_transformers.enable('json')

Loading BokehJS ...

DataTransformerRegistry.enable('json')

In [2]:
# Load in the genotype file, call it df_gt for genotype DataFrame
df_gt = pd.read_csv("../data/fish_activity_for_validation/150717_2A_genotype_3.txt", 
                  delimiter='\t', comment ='#')

In [3]:
# Tidy the DataFrame
df_gt = pd.melt(df_gt, var_name='genotype', value_name='location')

# Drop all rows that have a NaN in them
df_gt = df_gt.dropna()

df_gt = df_gt.reset_index(drop=True)

df_gt.loc[:,'location'] = df_gt.loc[:, 'location'].astype(int)

In [4]:
df_gt.head(10)

,genotype,location
0,wt,1
1,wt,3
2,wt,8
3,wt,12
4,wt,21
5,wt,28
6,wt,31
7,wt,33
8,wt,34
9,wt,35


In [5]:
# Now, load in the CSV file
fname = '../data/fish_activity_for_validation/150717_2A_2B.csv'
df = pd.read_csv(fname, comment='#')

First, we want to test that all the columns are what we excpect them to be before we start tidying the data.

In [6]:
def test_column_names(df, fname):
    """Ensure DataFrame has proper columns."""
    column_names = ['location','animal','user','sn','an','datatype','start','end','startreason',
                    'endreason','frect', 'fredur', 'midct', 'middur', 'burct', 'burdur', 'stdate',
                    'sttime']

    assert list(df.columns) == column_names, fname + ' has wrong column names.'

In [7]:
test_column_names(df, fname)

Now, we get rid of the c from location column, and convert to an integer type so we can merge this dataframe with the genotype dataframe.

In [8]:
sanititize_loc = lambda x: int(x.replace("c", ""))

df['location'] = df['location'].apply(sanititize_loc)

In [9]:
# We perform the merge.
df = pd.merge(df, df_gt)

We are only looking for fish in instrument 2A, numbered 1 through 96 in the activity data file, so we can get rid of fish in instrument 2B, numbered 97 and above, as they are not used.

In [10]:
df.head()

,location,animal,user,sn,an,datatype,start,end,startreason,endreason,frect,fredur,midct,middur,burct,burdur,stdate,sttime,genotype
0,1,z001,ZEBRALAB02\zebralab_user,1,0,quant,0.0,60.0,Beginning of session,End of period,0,60.0,0,0.0,0,0.0,17/07/2015,14:29:59,wt
1,1,z001,ZEBRALAB02\zebralab_user,1,0,quant,60.0,120.0,End of period,End of period,0,60.0,0,0.0,0,0.0,17/07/2015,14:30:59,wt
2,1,z001,ZEBRALAB02\zebralab_user,1,0,quant,120.0,180.0,End of period,End of period,6,59.0,6,0.9,0,0.0,17/07/2015,14:31:59,wt
3,1,z001,ZEBRALAB02\zebralab_user,1,0,quant,180.0,240.0,End of period,End of period,0,60.0,0,0.0,0,0.0,17/07/2015,14:32:59,wt
4,1,z001,ZEBRALAB02\zebralab_user,1,0,quant,240.0,300.0,End of period,End of period,0,60.0,0,0.0,0,0.0,17/07/2015,14:33:59,wt


In [11]:
df["location"] = df[df["location"] < 97]
df.tail()

,location,animal,user,sn,an,datatype,start,end,startreason,endreason,frect,fredur,midct,middur,burct,burdur,stdate,sttime,genotype
325396,96,z096,ZEBRALAB02\zebralab_user,1,0,quant,246840.0,246900.0,End of period,End of period,4,59.6,4,0.4,0,0.0,20/07/2015,11:03:59,het
325397,96,z096,ZEBRALAB02\zebralab_user,1,0,quant,246900.0,246960.0,End of period,End of period,28,55.2,29,4.8,0,0.0,20/07/2015,11:04:59,het
325398,96,z096,ZEBRALAB02\zebralab_user,1,0,quant,246960.0,247020.0,End of period,End of period,69,50.2,68,9.9,0,0.0,20/07/2015,11:05:59,het
325399,96,z096,ZEBRALAB02\zebralab_user,1,0,quant,247020.0,247080.0,End of period,End of period,62,51.6,63,8.4,0,0.0,20/07/2015,11:06:59,het
325400,96,z096,ZEBRALAB02\zebralab_user,1,0,quant,247080.0,247103.7,End of period,End of session,27,19.3,26,4.4,0,0.0,20/07/2015,11:07:59,het


We want to check that the 96 fish that we *should* have left are present and accounted for.

In [12]:
def check_num_fish(df, num_fish):
    
    fish_array = list(set(df["location"].values))

    if len(fish_array) != num_fish: 
        lost_fish = list(set(range(1, num_fish + 1)) - set(fish_array))
        
        return"Oh no! We've lost {} fishies at indicies: {}. We should have {} but we have {}!" \
                    .format(len(lost_fish), lost_fish, num_fish, num_fish - len(lost_fish))

    return "All fishies are accounted for! Thank the lord!"

In [13]:
check_num_fish(df, 96)

"Oh no! We've lost 17 fishies at indicies: [64, 65, 66, 67, 36, 68, 69, 70, 11, 47, 51, 83, 84, 22, 55, 62, 95]. We should have 96 but we have 79!"

Yikes! Unfortunately we are missed a good chunk of fish! This is good information to know.

We don't want any of our numerical columns to be negative; we'll write a general function for it and apply it to all the numerical columns.

In [14]:
# Loop through all the numerical columns (given in cols) and check that they are positive
def num_negative_col(df, cols):
    results = []
    for col in cols:
       results.append("The column {} has {} negative values".format (col, (df[col] < 0).sum()))
    return results

In [15]:
cols = ["location", "sn", "an", "start", "end", "frect", "fredur", "midct", "burct", "burdur"]
num_negative_col(df, cols)

['The column location has 0 negative values',
 'The column sn has 0 negative values',
 'The column an has 0 negative values',
 'The column start has 0 negative values',
 'The column end has 0 negative values',
 'The column frect has 0 negative values',
 'The column fredur has 0 negative values',
 'The column midct has 0 negative values',
 'The column burct has 0 negative values',
 'The column burdur has 0 negative values']

We want to make sure that all of our start date values in the stdate column are correct. This means the month, day, and year values need to be plausible. We check this with lambda expressions. (There's no obvious simple way to do this becuase sadly the date and time values are strings and not datetime objects).

In [101]:
def check_date(df):

    check_day = lambda x: isinstance(int(x[:2]), int) and int(x[:2]) <= 31 
    check_month = lambda x: isinstance(int(x[3:5]), int) and int(x[3:5]) <= 12 
    check_year = lambda x: isinstance(int(x[6:]), int) and int(x[6:]) < 2019
    df_t = df
    df_t["hi"] = df['stdate'].apply(check_date and check_month and check_year)
    return ("There are {} incorrectly formatted dates!" \
        .format(df_t[df_t['hi'] == False].shape[0]))

In [102]:
check_date(df)

'There are 0 incorrectly formatted dates!'

We now check the validity of times!

In [99]:
def check_time(df):

    check_hours = lambda x: isinstance(int(x[:2]), int) and int(x[:2]) <= 24 
    check_minutes = lambda x: isinstance(int(x[3:5]), int) and int(x[3:5]) <= 60 
    check_seconds = lambda x: isinstance(int(x[6:]), int) and int(x[6:]) <= 60
    df_t = df
    df_t["hi"] = df['sttime'].apply(check_hours and check_minutes and check_seconds)
    return ("There are {} incorrectly formatted dates!" \
        .format(df_t[df_t['hi'] == False].shape[0]))

In [100]:
check_time(df)

'There are 0 incorrectly formatted dates!'

We want to check that the duration of all the periods are equal! We take the most common interval duration and see how many data points deviate from the mode. We want this to fail silently becuase it is probable that we will have some outlier fish with longer or shorter intervals.

In [16]:
# Make sure that all the time intervals are equal
def test_time_intervals(df, verbose):
    df_temp = df
    df_temp["length"] = df["end"] - df["start"]
    results = []
    
    # Find the mode of the column
    mode = df_temp["length"].mode()[0]
    
    outliers = df_temp.loc[df_temp["length"] != mode]
    
    results.append("The mode is {}".format(mode))
    results.append("{} rows deviate from the mode".format(outliers.shape[0]))
    
    # We only want to print out all the information for each individual fish 
    # if the user requests it to reduce the number of output rows.
    if verbose:
        for index, row in outliers.iterrows():
            results.append("The row at index {} has a duration of {}"
                           .format(index, round(row["length"], 2)))

    return results 

In [17]:
test_time_intervals(df, True)

['The mode is 60.0',
 '237 rows deviate from the mode',
 'The row at index 3229 has a duration of 76.6',
 'The row at index 3230 has a duration of 43.4',
 'The row at index 4118 has a duration of 20.2',
 'The row at index 7348 has a duration of 76.6',
 'The row at index 7349 has a duration of 43.4',
 'The row at index 8237 has a duration of 20.2',
 'The row at index 11467 has a duration of 76.6',
 'The row at index 11468 has a duration of 43.4',
 'The row at index 12356 has a duration of 20.2',
 'The row at index 15586 has a duration of 76.6',
 'The row at index 15587 has a duration of 43.4',
 'The row at index 16475 has a duration of 20.2',
 'The row at index 19705 has a duration of 76.6',
 'The row at index 19706 has a duration of 43.4',
 'The row at index 20594 has a duration of 20.2',
 'The row at index 23824 has a duration of 76.6',
 'The row at index 23825 has a duration of 43.4',
 'The row at index 24713 has a duration of 22.0',
 'The row at index 27943 has a duration of 76.6',


As you can see, a lot of fish have a duration different than the mode of 60 seconds. This would be important if we were to do data analysis on the fish's activity.  

The middur column is what we use for computing activity. We will first check the activity column middur. Obviously, fish cannot be spend more seconds active than the total duration of the period.

We see that there are 3 discrepancies per fish. Based on the trend of index numbers, we can assume that the start and the end of each session were of different durations than the bulk of the experimental data. We also see that indicies 40300 and 40301 have unique durations of 65.5 and 54.5. We hypothesize that that this is due to one period accidentally going long and the next period being cut short to compensate. 

Obviously, we would need to talk to the expirementer for accurate explanations, but data validation is helful in helping us know what questions to ask.

In [18]:
def test_middur(df):
    df_temp = df
    df_temp["length"] = df["end"] - df["start"]
    
    outliers = df_temp[df["middur"] > df_temp["length"]].shape[0]
    
    if outliers != 0:
        return "Something is wrong! In {} periods, fish ".format(outliers) + \
                "were active longer than the total duration"
    
    return "Number of seconds active is less than total period duration for all trials"

In [19]:
test_middur(df)

'Number of seconds active is less than total period duration for all trials'

We now know that are activity values are within the realm of possibility! 

Let's make sure that all the fish have the same number of data points. Once again, we want this to fail silently becuase it is probable that we will have some outlier fish with more or less datapoints.

In [20]:
def test_check_num_datapoints(df):
    NUM_POINTS_PER_FISH = 4119 # We know each fish should have 4119 data points
    
    fish_set = set(df["location"].values)
    results = []
    
    # Let's find out how many data points each fish has
    for fish in fish_set:
        fish_count = df[df["location"] == fish].shape[0]
        if fish_count != NUM_POINTS_PER_FISH:
            results.append("Fish number {} has {} points instead of {}" \
                           .format(fish, fish_count, NUM_POINTS_PER_FISH))
    if len(results) == 0:
        results.append("All the fish have {} datapoints! No need to fret!" \
                       .format(NUM_POINTS_PER_FISH))
    return results

In [21]:
test_check_num_datapoints(df)

['All the fish have 4119 datapoints! No need to fret!']

Now that we know that all the fish have the same number of data points, we can check that only the first entry of each fish has start reason 'Beginning of session' and only the last entry of each fish has end reason "End of session"

In [55]:
def test_start_end_reason2(df):
    results = []
    counter = 0
    
    # Assume that the data is in the form that we have validated above:
    # each fish has an equal number of data points!
    
    num_entries_per_fish = df[df["location"] == 1].shape[0]
    
    for index, row in df.iterrows():
        sys.stdout.write("\r%.2f percent done."%(100*index / len(df.index)))
        # all start reasons but the first of each fish should be "End of Period"
        
        # if we aren't at the first of a fish
        if index % num_entries_per_fish != 0:
            if df["startreason"].iloc[index] != 'End of period':
                results.append("Row at index {} has an invalid startreason of {}" \
                               .format(index, df["startreason"].iloc[index]))
                
        # else we are at the start of a fish
        else:
            if df["startreason"].iloc[index] != 'Beginning of session':
                results.append("Missing the beginning of session at index {}" \
                               .format(index))
                
        # all end reasons but the last of each fish should be "End of Period"
        
        # we aren't at the end of a fish
        if index % num_entries_per_fish != num_entries_per_fish - 1:
            # print ("i: {}, {}, {}, {}".format(index, index % 4118, index % 4118 != counter, counter))
            if df["endreason"].iloc[index] != "End of period":
                results.append("Row at index {} has an invalid endreason of {}" \
                               .format(index, df["endreason"].iloc[index] ))
        
        # we are at the end of the fish
        else:
            if df["endreason"].iloc[index] != 'End of session':
                results.append("Missing the end of session at index {}" \
                               .format(index))
            counter += 1
            
    if len(results) == 0:
        results.append("The start and the end reasons for all the data points look good!")
    
    sys.stdout.write("\r100 percent done.\n")
    
    return results
            

In [56]:
test_start_end_reason2(df)

4119
100 percent done.


['The start and the end reasons for all the data points look good!']

In [43]:
df.head()

,location,animal,user,sn,an,datatype,start,end,startreason,endreason,frect,fredur,midct,middur,burct,burdur,stdate,sttime,genotype,length
0,1,z001,ZEBRALAB02\zebralab_user,1,0,quant,0.0,60.0,Beginning of session,End of period,0,60.0,0,0.0,0,0.0,17/07/2015,14:29:59,wt,60.0
1,1,z001,ZEBRALAB02\zebralab_user,1,0,quant,60.0,120.0,End of period,End of period,0,60.0,0,0.0,0,0.0,17/07/2015,14:30:59,wt,60.0
2,1,z001,ZEBRALAB02\zebralab_user,1,0,quant,120.0,180.0,End of period,End of period,6,59.0,6,0.9,0,0.0,17/07/2015,14:31:59,wt,60.0
3,1,z001,ZEBRALAB02\zebralab_user,1,0,quant,180.0,240.0,End of period,End of period,0,60.0,0,0.0,0,0.0,17/07/2015,14:32:59,wt,60.0
4,1,z001,ZEBRALAB02\zebralab_user,1,0,quant,240.0,300.0,End of period,End of period,0,60.0,0,0.0,0,0.0,17/07/2015,14:33:59,wt,60.0
